## MotionCommander Basics

Reference Documentation: 
* [Step-by-Step: Motion Commander](https://www.bitcraze.io/documentation/repository/crazyflie-lib-python/master/user-guides/sbs_motion_commander/)
* [MotionCommander GitHub](https://github.com/bitcraze/crazyflie-lib-python/blob/master/examples/step-by-step/sbs_motion_commander.py)


In [ ]:
import logging
import sys
import time
from threading import Event

# Crazyflie imports
import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.positioning.motion_commander import MotionCommander
from cflib.utils import uri_helper

### Global Variables & Configs

In [25]:
# Your radio frequency. This can be found in the Crazyflie client by running "cfclient" in your terminal.
URI = uri_helper.uri_from_env(default='radio://0/80/2M/E7E7E7E7E8')

# --- Global variables
DEFAULT_HEIGHT = 1.0
BOX_LIMIT = 0.5

# Event to signal that the flow deck is attached
deck_attached_event = Event()

# Variable to store the (x, y) position estimates used by the logger
position_estimate = [0, 0]

# Only output errors from the logging framework
logging.basicConfig(level=logging.ERROR)

In [31]:
def param_deck_flow(_, value_str):
    """
    Function to check if the flow deck is attached to the drone. 
    Note: that the value type that the param_deck_flow() is a string type, so we convert it to an int before operations.
    All deck values are here: https://www.bitcraze.io/documentation/repository/crazyflie-firmware/master/api/params/#deck
    """
    value = int(value_str)
    print(value)
    if value:
        deck_attached_event.set()
        print('Deck is attached!')
    else:
        print('Deck is NOT attached!')


def take_off_simple(scf):
    """
    Simple take off and land function using the MotionCommander class.
    1. Take off to a default height (defined in the global variable DEFAULT_HEIGHT)
    2. Move up 0.3m
    3. Hover for 3 seconds
    4. Land
    """
    with MotionCommander(scf, default_height=DEFAULT_HEIGHT) as mc:
        # Take off to the default height and then go up 0.3m
        mc.up(0.3)
        time.sleep(3)
        mc.stop()
        

def move_linear_simple(scf):
    """
    Simple linear movement function using the MotionCommander class.
    1. Take off to a default height (defined in the global variable DEFAULT_HEIGHT)
    2. Move forward 0.5m
    3. Move backward 0.5m
    4. Land
    """
    with MotionCommander(scf, default_height=DEFAULT_HEIGHT) as mc:
        time.sleep(1)
        mc.forward(0.5)
        time.sleep(1)
        mc.back(0.5)
        time.sleep(1)


def log_pos_callback(timestamp, data, logconf):
    # Prints logged X, Y position estimates
    print(data)



In [30]:
if __name__ == '__main__':
    cflib.crtp.init_drivers()
    
        
    with SyncCrazyflie(URI, cf=Crazyflie(rw_cache='./cache')) as scf:
        # Callback to check if the flow deck is attached
        scf.cf.param.add_update_callback(group='deck', name='bcFlow2', cb=param_deck_flow)

        # Log event actions
        logconf = LogConfig(name='Position', period_in_ms=10)
        logconf.add_variable('stateEstimate.x', 'float')
        logconf.add_variable('stateEstimate.y', 'float')
        scf.cf.log.add_config(logconf)
        logconf.data_received_cb.add_callback(log_pos_callback)
        
        
        # Wait until the flow deck is attached (max wait time is 5 seconds)
        if not deck_attached_event.wait(timeout=5):
            print('No flow deck detected!')
            sys.exit(1)
            
        logconf.start()
            
        # Arm the Crazyflie
        scf.cf.platform.send_arming_request(True)
        time.sleep(1.0)
        move_linear_simple(scf)
        
        logconf.stop()


{'stateEstimate.x': 0.05608847364783287, 'stateEstimate.y': 0.00872065033763647}
{'stateEstimate.x': 0.056340303272008896, 'stateEstimate.y': 0.00928527768701315}
{'stateEstimate.x': 0.05632936581969261, 'stateEstimate.y': 0.008074657991528511}
{'stateEstimate.x': 0.0563211664557457, 'stateEstimate.y': 0.007620169315487146}
{'stateEstimate.x': 0.05582382157444954, 'stateEstimate.y': 0.007902424782514572}
1
Deck is attached!
{'stateEstimate.x': 0.0558166541159153, 'stateEstimate.y': 0.007698849309235811}
{'stateEstimate.x': 0.056330204010009766, 'stateEstimate.y': 0.00892542488873005}
{'stateEstimate.x': 0.05608189478516579, 'stateEstimate.y': 0.013723003678023815}
{'stateEstimate.x': 0.05606949329376221, 'stateEstimate.y': 0.018456047400832176}
{'stateEstimate.x': 0.055593930184841156, 'stateEstimate.y': 0.01864263042807579}
{'stateEstimate.x': 0.05559326335787773, 'stateEstimate.y': 0.018140694126486778}
{'stateEstimate.x': 0.05604999512434006, 'stateEstimate.y': 0.018615642562508583}